# NZU price data scraping

To scrape the data using a Python package called Beautiful Soup, we need to first use Python's `request` module to get the entire HTML source of a given website. We are interested in two URLs:
```
https://www.carbonnews.co.nz/tag.asp?tag=Jarden+NZ+Market+Report
https://www.carbonnews.co.nz/tagarchive.asp?tag=Jarden+NZ+Market+Report
```
So let us get and parse them separately, and store them as two beautiful soups.

In [1]:
import requests
from bs4 import BeautifulSoup

url="https://www.carbonnews.co.nz/"

page = requests.get(url+"tag.asp?tag=Jarden+NZ+Market+Report")
soup = BeautifulSoup(page.content, "html.parser")

page = requests.get(url+"tagarchive.asp?tag=Jarden+NZ+Market+Report")
soup2 = BeautifulSoup(page.content, "html.parser") 

Using our knowledge of the underlying HTML structure, we can pick out the headline and summary elements for each story, in both soups, and store the extracted elements in lists.

In [2]:
# Find all the h1, h2, and h3 headlines in first soup
helements = soup.find_all(["h1","h2","h3"], class_="Headline")
# Find the h3 headlines in second soup
helements+= soup2.find_all("h3", class_="Headline")

# Find all the accompanying summaries
pelements = soup.find_all("p", class_=None ) # the h1 headline
pelements+= soup.find_all("p", class_=["StoryIntro","StoryIntro_small"]) # h2 and h3 headlines
pelements+= soup2.find_all("p", class_="StoryIntro_small") # h3 headlines from the second soup

# Find all h4 headlines in the second soup
helements_arch = soup2.find_all(["h4"], class_="Headline")

Note that `helements` and `pelements` should be commensurate, while `helements_archive` should be much longer. 

In [3]:
print(len(helements), len(pelements))
print(len(helements_arch))
print(helements_arch[0])

39 39
2764
<h4 class="Headline" xstyle="font-weight:normal;"><img alt="" height="8" src="images/arrow.gif" width="8"> 14 Jun 23  <a href="story.asp?storyID=27947">ETS auction fails to clear</a></img></h4>


Let us now extract just the three relevant bits of text for each and every story: the headline string, the date string, and the href string.

In [4]:
headlines = []; datestrings = []; hrefs = []

# First loop over more recent headlines with summaries
for i in range(len(helements)):
    body = helements[i].find("a")
    headlines.append(body.text.strip())
    hrefs.append(body.get("href"))
    # The date string is the first text before the ' - ' in the summary.
    datestrings.append(pelements[i].text.strip().split(' - ')[0])
        
# Then loop over older archived headlines without summaries
for h in helements_arch:
    body = h.find("a")
    headline = body.text.strip()
    headlines.append(headline)
    hrefs.append(body.get("href"))
    # The date string is the text that's not part of the headline 
    datestrings.append(h.text.strip().replace(headline,''))

Check that the three lists are of the same length, and then print the first few and the last few entries in each list, just for illustration.

In [5]:
print(len(headlines), len(datestrings), len(hrefs),'\n')

for i in range(8):
    print(datestrings[i],' | ',headlines[i],' | ',hrefs[i])

print()

for i in range(1,9):
    print(datestrings[-i],' | ',headlines[-i],' | ',hrefs[-i])

2803 2803 2803 

Thursday  |  MARKET LATEST: NZUs $60.75  |  story.asp?storyID=28419
Wednesday  |  MARKET LATEST: NZUs $60.25  |  story.asp?storyID=28409
Tuesday  |  MARKET LATEST: NZUs $59.90  |  story.asp?storyID=28396
Monday  |  MARKET LATEST: NZUs $58.00  |  story.asp?storyID=28382
4 Aug 23  |  MARKET LATEST: NZUs $58.00  |  story.asp?storyID=28371
3 Aug 23  |  MARKET LATEST: NZUs $57.00  |  story.asp?storyID=28360
2 Aug 23  |  MARKET LATEST: NZUs $57.50  |  story.asp?storyID=28345
1 Aug 23  |  MARKET LATEST: NZUs $59.75  |  story.asp?storyID=28334

13 Jun 08    |  Current carbon credits available  |  story.asp?storyID=1194
13 Jun 08    |  Latest strip of CERs   2008  2012 vintage  indicative mid prices  |  story.asp?storyID=1193
17 Jun 08    |  Current Carbon Credits Available  |  story.asp?storyID=1219
17 Jun 08    |  Latest Strip of CERs   2008  2012 Vintage  Indicative Mid Prices  |  story.asp?storyID=1218
17 Jun 08    |  Oil pushes carbon higher  |  story.asp?storyID=121

Convert date-strings to date objects using the `datetime` module. For later reuse, I implemented the conversion as a function called `strings2dates` inside the `functions.py` file.

In [6]:
from scraping_functions import strings2dates
dates = strings2dates(datestrings)

We know that the price of NZUs is first reported on 14 May 2010, so we can discard older stories from our lists.

In [7]:
from datetime import datetime
while dates[-1] < datetime(2010,5,14).date():
    dates.pop()
    datestrings.pop()
    headlines.pop()
    hrefs.pop()

Print the date, headline, and hrefs for oldest few stories still  remaining in the list, just for illustration.

In [8]:
for i in range(1,9):
    print(dates[-i],' | ',headlines[-i],' | ',hrefs[-i])

2010-05-14  |  NZUs have quiet week trading in the $17s  |  story.asp?storyID=4529
2010-05-21  |  Carbon trading in the high teens  |  story.asp?storyID=4540
2010-05-28  |  Carbon price consolidates  |  story.asp?storyID=4551
2010-06-11  |  Carbon moves up to mid $17s  |  story.asp?storyID=4588
2010-06-25  |  Demand builds in local market  |  story.asp?storyID=4623
2010-07-02  |  Carbon price firm, but supply is short  |  story.asp?storyID=4643
2010-07-09  |  Sellers few and far between  |  story.asp?storyID=4658
2010-07-16  |  Market firm but drop on the way  |  story.asp?storyID=4675


Now print all the remaining stories' date, headline, and URL to a CSV file called `headline_scrape.csv`, with the stories ordered from oldest to latest.

In [9]:
import csv
url = 'https://www.carbonnews.co.nz/'
with open('headline_scrape.csv', 'w', newline='') as file:    
    writer = csv.writer(file)
    writer.writerow(['date','headline','url'])
    for i in range(1,1+len(dates)):
        writer.writerow([dates[-i],headlines[-i],url+hrefs[-i]])        

In [10]:
def parse_headline(headling)
"""
Function taking a list of strings and returning a list of dollar-valued prices
extracted from each string.

INPUTS:
------

headline: a string with headline text

OUTPUTS:
-------

price: a real-valued price extracted from headline
"""

words = headline.split()
keywords = ['NZU', 'settles']

price = None
if('MARKET LATEST' in headline):
    pricestring = words[-1].strip('$')
    try:
        price = float(pricestring)
    except ValueError:
        print("WARNING: '{}' is not a float!".format(pricestring))
elif():

    

SyntaxError: expected ':' (3482318710.py, line 1)

Furthermore, the date is not actually included in the headlines, but it nonetheless can still be scraped from just the two main web pages, though the date formatting there makes it more awkward to parse than when scraping from individual story pages. 

So, for demonstration purposes, let us take a multi-prong approach with some redundancies, and we will two approaches using different toolsets:
- [Bash](https://www.gnu.org/software/bash/) and Linux command-line tools to scrape just the spot price from the headlines; and
- Python package called [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/) to get the spot prices from the headings, and then also look for the headline price in the story's first paragraph. 


## Command-line approach

We are interested in two URLs:
```
https://www.carbonnews.co.nz/tag.asp?tag=Jarden+NZ+Market+Report
https://www.carbonnews.co.nz/tagarchive.asp?tag=Jarden+NZ+Market+Report
```

First download the HTML source using `wget`, then use `grep`, `sed`, and `awk` to get the values.

## Beautiful Soup approach

All CarbonNews stories can be accessed via public URLs such as 
```
https://www.carbonnews.co.nz/story.asp?storyID=28274
```
for the story pictured above. Given such a URL, we can obtain the page HTML code and scrape the required information from that. Conveniently, the HTML source follows a fairly simple structure, and the entire text snippet can be extracted by   

In [11]:
def check(headline):
    
    if('MARKET LATEST:' in headline or 
       ('NZU' in headline and headline.count('$') == 1)):
        result = True
    else:
        result = False
    
    return result

In [12]:
def parse_stories(hrefs):
    
    import requests
    from bs4 import BeautifulSoup
    from datetime import datetime
    
    data = []
    url_home="https://www.carbonnews.co.nz/"
    
    for href in hrefs:
        
        url = url_home+href
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        
        # Process the heading, which should be the first 'h1' element in soup
        h1elements = soup.find_all("h1", class_="story")
        heading = h1elements[0].text.strip()
        prices = [word.strip('$.') for word in heading.split() if word[0] == '$']
        if len(prices) == 1:
            price = prices[0]
        else:
            price = 'NaN'
            if len(price) > 1:
                print('WARNING: Heading contains multiple dollar values.')
            else:
                print('WARNING: Heading contains no dollar values.')        
            
        
        # Process text snippet, which should be the only div element of class StoryFirstPara in soup
        para = soup.find("div", class_="StoryFirstPara").text.strip()
        prices = [word.strip('$.') for word in para.split() if word[0] == '$']
        if price not in prices:
            print('WARNING: Heading price not in story text snippet.')
        
        # Process the story date
        h4elements = soup.find_all("h4")
        date = h4elements[0].text.strip()        
        if date.split()[0] == 'Today':
            date = datetime.today().strftime('%Y-%m-%d')
        else:
            date = ' '.join(date.split()[1:-1]) # strip weekday and time
            date = datetime.strptime(date,'%d %b %y').strftime('%Y-%m-%d')
        
        data.append({'date':date, 'price':price, 'url':url})
    
    return data